In [1]:
# !pip install noisereduce
# !pip install phonemizer
# !pat upadte & apt install -y espeak

In [2]:
from inference import StyleTTS2

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
import os
import sys
import phonemizer
import torch

import librosa
import IPython.display as ipd

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
def get_phoneme(text, lang):
    try:
        my_phonemizer = phonemizer.backend.EspeakBackend(language=lang, preserve_punctuation=True,  with_stress=True, language_switch='remove-flags')
        return my_phonemizer.phonemize([text])[0]
    except Exception as e:
        print(e)

In [6]:
text = 'Hola, necesito ayuda con mi plan de celuar'
get_phoneme(text, 'es-la')

'ˈola, nˌesesˈito ajjˈuða kɔn mi plˈan de sˌeluˈaɾ '

# Load models

In [259]:
config_path='Configs/config.yaml'
# models_path='Models/Finetune/epoch_00013.pth' // OK
models_path='Models/Finetune/epoch_00025.pth'

In [260]:
model = StyleTTS2(config_path, models_path).eval().to(device)

/opt/conda/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/StyleTTS2-lite/inference.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. Th

decoder : 54289492
predictor : 16194612
text_encoder : 5606400
style_encoder : 13845440

Total : 89935944


# Synthesize speech

In [261]:
speaker = {
    "path": '/storage/services/tts_finetuning/notebooks/StyleTTS2_Finetune/Dataset_FT/wavs/1873_V.wav', # Ref audio path
    "speed": 1.3, # Speaking speed
}

In [262]:
max_samples = 24000 * 20 #max 20 seconds ref audio

wave, sr = librosa.load(speaker['path'], sr=24000)
audio, index = librosa.effects.trim(wave, top_db=30)
if sr != 24000:
    audio = librosa.resample(audio, sr, 24000)

if len(audio) > max_samples:
    audio = audio[:max_samples]
display(ipd.Audio(audio, rate=24000, normalize=True))

In [263]:
avg_style = True # True #BOOL   Split the ref audio and calculate the avg styles.
stabilize = False #BOOL   Stabilize speaking speed.
denoise = 0.3 #FLOAT  Adjust the strength of the denoiser. Value range is [0, 1]
n_merge = 16 # 16 

In [285]:
text = 'Hola, necesito ayuda con mi plan de celular. Y te invito a diligenciar esta encuesta, muchas gracias'
# text = '''
# ¡Hola! ¡Que alegría tenerte por aqui! Soy tu asistente virtual, siempre listo para ayudarte en lo que necesites. ¿Como estás hoy? Espero que tengas un dia increible lleno de energía y buenas vibras. Estoy aquí para hacerte la vida mas fácil. asi que no dudes en preguntarme cualquier cosa. Ya sea que necesites ayuda con un problema técnico. quieras aprender algo nuevo o simplemente quieras charlar un rato, ¡estoy a tu disposición! Recuerda que no hay preguntas tontas, así que siéntete libre de explorar y consultar todo lo que se te ocurra. Juntos podemos resolver cualquier desafío que se presente. ¡Gracias por confiar en mi! Estoy emocionado de acompañarte en este viaje y ayudarte a alcanzar tus metas. ¿En qué puedo ayudarte hoy?
# '''.strip()

In [286]:
with torch.no_grad():
    styles = model.get_styles(speaker, denoise, avg_style)

Computing the style for: /storage/services/tts_finetuning/notebooks/StyleTTS2_Finetune/Dataset_FT/wavs/1873_V.wav


In [289]:
%%time
with torch.no_grad():
    phonemes = get_phoneme(text=text, lang="es-la")
    r = model.generate(phonemes, styles, stabilize, n_merge)

CPU times: user 6.2 s, sys: 201 ms, total: 6.4 s
Wall time: 112 ms


In [288]:
# print('Synthesized:')
display(ipd.Audio(r, rate=24000, normalize=True))

# Output

In [257]:
from scipy.io.wavfile import write

In [258]:
idx = len(os.listdir('SampleOutput')) + 1
output_path = os.path.join('SampleOutput', f'sample_{idx}.wav')
print(f'Generated: {output_path}')
write(output_path, 24000, r)

Generated: SampleOutput/sample_5.wav


In [203]:
write(os.path.join('SampleOutput', 'reference.wav'), 24000, audio)